# Workflow 4

### Import est_lib Functions

In [1]:
from est_lib.util.obspy_util import *
from est_lib.util.obspy_plot import ray_plot

### Import Some other est_lib Functions Used in this Workflow

In [39]:
import torch
from est_lib.nn.cp_line_simple import cp_line_simple
from est_lib.dataset.seismic_dataset_new import EQDataset, get_device

### Import Some other Functions that are also used in this Workflow

In [3]:
import os

### Specify Search Parameters

In [4]:
station_list = ['HOLB','PACB','WOSB','HOPB']
channel_list = ['HHE','HHN','HHZ']

# Note: channel_list specifies the channels of interest
#    In this example, we retrieve velocity data for 
#    all three directions (E, N and Z). To get acceleration
#    data replace with ['HNE','HNN','HNZ']

'''
Specify a Time of interest. Note that that the timestamp
is in UTC.
'''
event_time = "2019-12-25T03:36:01.578000Z"
event_lat = 50.6081
event_lon = -129.9656
event = ('6.3',event_lat,event_lon,event_time)

### Retrieve Station Metadata

In [5]:
'''
Use the inventory_retriever function to retrieve
metadata about stations in station_list
'''
station_metadata = inventory_retriever(sta_list=station_list)

In [6]:
print(station_metadata)

Inventory created at 2021-08-06T03:09:25.000000Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.47
		    http://service.iris.edu/fdsnws/station/1/query?network=CN&station=H...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (1):
			CN
		Stations (4):
			CN.HOLB (Holberg, BC, CA)
			CN.HOPB (Hope, BC, CA)
			CN.PACB (Port Alice, BC, CA)
			CN.WOSB (Woss, BC, CA)
		Channels (65):
			CN.HOLB..EHZ, CN.HOLB..HHZ (4x), CN.HOLB..HHN (4x), 
			CN.HOLB..HHE (4x), CN.HOLB..HNZ (3x), CN.HOLB..HNN (3x), 
			CN.HOLB..HNE (3x), CN.HOPB..BHZ (2x), CN.HOPB..BHN (2x), 
			CN.HOPB..BHE (2x), CN.HOPB..HHZ (2x), CN.HOPB..HHN (2x), 
			CN.HOPB..HHE (2x), CN.HOPB..HNZ (2x), CN.HOPB..HNN (2x), 
			CN.HOPB..HNE (2x), CN.PACB..HHZ (2x), CN.PACB..HHN (2x), 
			CN.PACB..HHE (2x), CN.PACB..HNZ (2x), CN.PACB..HNN (2x), 
			CN.PACB..HNE (2x), CN.WOSB..EHZ, CN.WOSB..HHZ, CN.WOSB..HHN, 
			CN.WOSB..HHE, CN.WOSB..HNZ (3x), CN.WOSB..HNN (3x), 
			CN.WOSB..HNE (3x)


### Retrieve Seismic Streams

In [7]:
'''
Use the stream_retriever function to retrieve
seismic streams from an IRIS' FDSN compliant
datacenter.

We instruct the function to fetch data starting
from 500 seconds before the specified time upto
150 seconds after.

That is, if the time of interest is at 0, we
fetch data from -500 to 1500.
'''

seconds_before = 500
seconds_after = 1500

'''
Datasets for supervised deep learning have a training
set and a test set. Let us split the data we retrieve
such that data related to all but the last station (HOPB)
in station_list forms the training set and the data from
HOPB forms the test set.
'''

train_data = stream_retriever(event_time=event_time,
                        time_format='string',
                        seconds_before = 500,
                        seconds_after = 1500,
                        sta_list = station_list[0:-1],
                        channel_list = channel_list)

test_data = stream_retriever(event_time=event_time,
                        time_format='string',
                        seconds_before = 500,
                        seconds_after = 1500,
                        sta_list = station_list[-1:],
                        channel_list = channel_list)

### Write Streams to Local Filesystem

#### Specify the Output Directory

In [9]:
filepath = os.path.abspath(os.path.join('./Data'))
if(not os.path.isdir(filepath)):
    os.mkdir(filepath)

In [10]:
print(filepath)

C:\Users\aksha\Desktop\eew-spatio-temporal\workflow_demos\Data


#### Write Train File to File System

In [11]:
train_x_f_path = os.path.join(filepath,'train_x.npy')
train_x_file = stream_data_writer(train_data,
                              station_metadata,
                              train_x_f_path,
                              station_list[0:-1],
                              channel_list)

#### Write Test File to File System

In [12]:
train_y_f_path = os.path.join(filepath,'train_y.npy')
train_y_file = stream_data_writer(test_data,
                              station_metadata,
                              train_y_f_path,
                              station_list[-1:],
                              channel_list)

### Write Inventory Data to Local Filesystem

In [13]:
'''
Use inventory_writer to save station_metadata to a
file in the local filesystem in the XML format.
Specify a path to the new file as the second argument.
'''
metadata_file = os.path.join(filepath,'demo_stations_metadata.xml')
inventory_writer(station_metadata,metadata_file)

'C:\\Users\\aksha\\Desktop\\eew-spatio-temporal\\workflow_demos\\Data\\demo_stations_metadata.xml'

### Initialize a PyTorch Dataset 

In [15]:
train_dataset = EQDataset(metadata_file,
                    train_x_file,train_y_file,
                    sta_list=station_list,
                    chan_list=channel_list,
                    ip_dim=len(channel_list),
                    num_nodes=len(station_list[0:-1]),
                    seq_length=500,horizon=2000)

### Initialize a Dataset Loader

In [31]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size=1)

### Initialize a Neural Network

In [40]:
net = cp_line_simple(num_in_nodes=len(station_list[0:-1]),
                     feat_size=len(channel_list)).to(get_device())

### Pass a Datapoint from the PyTorch Dataset into the Model

In [41]:
data_point = next(iter(train_data_loader))

In [42]:
net(data_point[0])

[tensor([[-0.4189]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.2631]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2421]], device='cuda:0', grad_fn=<AddmmBackward>)]

In [43]:
print(data_point[1])

tensor([[[ 508., -320., -300.]]], device='cuda:0')
